In [6]:
import os
from dotenv import load_dotenv
import json
from apify_client import ApifyClient
import duckdb as db

con = db.connect()
con.install_extension('json')
con.load_extension('json')

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [2]:
# Instala o python-dotenv se não estiver instalado
try:
    import dotenv
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])

In [3]:
# Initialize the ApifyClient with your API token from environment variable
client = ApifyClient(os.getenv("APIFY_API_TOKEN"))

# Prepare the Actor input
run_input = {
    "usernames": [
        "promodeamiga",
        "promosdarafa"
    ],
    "maxUsernames": 2,
    "delayBetweenRequests": 2000,
}

# Run the Actor and wait for it to finish
run = client.actor("7eNb0Yr5dLwy1jfPS").call(run_input=run_input)

# # Fetch and print Actor results from the run's dataset (if there are any)
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
#     print(item)

[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> Status: RUNNING, Message: 
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:34.242Z ACTOR: Pulling container image of build 6SdYWbsdK1dAUtt2P from registry.
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:34.244Z ACTOR: Creating container.
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:34.280Z ACTOR: Starting container.
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:35.007Z INFO  System info {"apifyVersion":"3.5.0","apifyClientVersion":"2.17.0","crawleeVersion":"3.15.1","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:36.726Z ✅ Stories checked for @promodeamiga → stories saved
[apify.instagram-story-viewer runId:ZDJtC18HA4anXIhT8] -> 2025-11-07T13:49:41.495Z ✅ Stories checked for @promosdarafa → stories saved
[apify.instagram-story-viewer runId:ZDJtC18HA

In [4]:
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    with open(f'output_{item["requested"]}.json', 'a', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)
        f.write('\n')

In [7]:
path = r'teste_json\*.json'
df = con.execute('''
with parte1 as (
SELECT
requested as username,
UNNEST(stories) as stories
FROM read_json_auto(?, ignore_errors=true)
),
parte2 as (
SELECT
username,
UNNEST(stories.story_link_stickers).story_link.display_url as story_link_url
FROM parte1
)   
SELECT username,
story_link_url,
case
when split(story_link_url, '/')[1] = 'amzlink.to' then 'Amazon'
when split(story_link_url, '/')[1] = 'mercadolivre.com' then 'Mercado Livre'
when split(story_link_url, '/')[1] = 's.shopee.com.br' then 'Shopee'
when split(story_link_url, '/')[1] = 'minhaloja.natura.com' then 'Natura'
when split(story_link_url, '/')[1] = 'magazinevoce.com.br' then 'Magazine Você'
when split(story_link_url, '/')[1] = 'elausa.com.br' then 'Ela Usa'
when split(story_link_url, '/')[1] = 'epocacosmeticos.com.br' then 'Época Cosméticos'
when split(story_link_url, '/')[1] = 'natura.com.br' then 'Natura'
when split(story_link_url, '/')[1] = 'sminhaloja.natura.com' then 'Natura'
else null end as origin
FROM parte2
''', [path]).df()
df.to_csv('output_final.csv', index=False)